<a href="https://colab.research.google.com/github/antoinebachand/Deep-Learnig-/blob/main/Binary_1target.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Neural Network for Prediction of Hydrogen Thermodynamics**
# **Binary with 1 Target**
### Author: Antoine Bachand (antoinebachand@outlook.com)

The training data set is obtained through a model adapted from Kushnir et al. (2012) for hydrogen storage. DOI: 10.1016/j.ijheatmasstransfer.2012.05.055 


In [302]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn.functional as functional
import torch.nn as nn

In [303]:
#load the data
df = pd.read_csv('data_10000.csv')
df.columns = ['Injection Temperature (K)', 'Intitial Temperature (K)', 'Total Mass (kg)', 'Injection Time (s)', 'Max Pressure (MPa)','Max Temperature (Celsius)']

The inputs to the neural network are:
- Initial rock temperature (K)
- Injection temperature (K)
- Total injection mass (kg)
- Injection time (s) 

and the outputs are as follows:
- Maximum pressure ( MPa )
- Maximum temperature ( °C )  

In [314]:
df.head()

,Injection Temperature (K),Intitial Temperature (K),Total Mass (kg),Injection Time (s),Max Pressure (MPa),Max Temperature (Celsius),Binary_Pressure
0,282.469062,300.763007,1869.140142,13607.354450,110.988812,67.340630,1
1,399.261432,306.867055,611.049819,14247.207020,24.467629,59.982319,0
2,332.949980,308.329176,1435.526958,5114.476186,79.346296,98.232711,1
3,398.578465,303.391355,1600.185040,15412.538030,92.198101,96.047796,1
4,329.299607,304.099858,871.039883,8019.916604,37.879257,66.882724,0


In [315]:
df.describe()

,Injection Temperature (K),Intitial Temperature (K),Total Mass (kg),Injection Time (s),Max Pressure (MPa),Max Temperature (Celsius),Binary_Pressure
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,373.359956,292.831066,1251.091858,13534.993797,70.186064,87.375692,0.605800
std,57.798521,11.607488,431.115254,5781.543880,61.757836,166.266974,0.488703
min,273.028357,273.005582,500.074091,3600.220147,16.635890,9.773542,0.000000
25%,323.171228,282.742639,877.214602,8496.583859,37.319443,52.352185,0.000000
50%,373.583463,292.672869,1255.380370,13463.256415,61.838304,70.354742,1.000000
75%,423.437581,302.876639,1626.478623,18582.071768,91.863710,93.249890,1.000000
max,472.990125,312.998830,1999.955495,23599.455440,2369.358069,8172.750350,1.000000


# Binary modification

Here we simplify the dataset to develop a first version of the neural network. We use only 1 target (the pressure) instead of two and we transform this last output in a binary form (0 when the pressure is lower than 50 MPa and 1 when it is higher than 50 MPa)

In [316]:
Binary_Pressure = []
for i in df['Max Pressure (MPa)']:
  if i < 50:
    Binary_Pressure.append(0)
  else:
    Binary_Pressure.append(1) 

df['Binary_Pressure'] = Binary_Pressure

df_drop = df.drop(['Max Pressure (MPa)','Max Temperature (Celsius)',],axis=1)

df_drop.head()

,Injection Temperature (K),Intitial Temperature (K),Total Mass (kg),Injection Time (s),Binary_Pressure
0,282.469062,300.763007,1869.140142,13607.354450,1
1,399.261432,306.867055,611.049819,14247.207020,0
2,332.949980,308.329176,1435.526958,5114.476186,1
3,398.578465,303.391355,1600.185040,15412.538030,1
4,329.299607,304.099858,871.039883,8019.916604,0


Since our different features are on different ranges, a normalization is applied.

In [317]:
# Normalization
norm = MinMaxScaler().fit(df_drop) 
nd_norm = norm.transform(df_drop)
df_norm = pd.DataFrame(nd_norm)

df_norm.columns = ['Injection Temperature (K)', 'Intitial Temperature (K)', 'Total Mass (kg)', 'Injection Time (s)', 'Max Pressure']


In [318]:
df_norm.describe()

,Injection Temperature (K),Intitial Temperature (K),Total Mass (kg),Injection Time (s),Max Pressure
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,0.501754,0.495721,0.500718,0.496758,0.605800
std,0.289048,0.290236,0.287433,0.289088,0.488703
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.250762,0.243468,0.251447,0.244828,0.000000
50%,0.502872,0.491765,0.503577,0.493171,1.000000
75%,0.752190,0.746903,0.750996,0.749121,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000


#Preparing data for training/testing

Here we split the data between training (80%) and testing (20%) in order to test the neural network with unseen data

In [319]:
# The columns are split between independent (Input) and dependent (Output)
columns = df_norm.columns.values.tolist()
# The independent ones correspond to the last two columns of the data set. 
independent = columns[:-1]
dependent = columns[-1]         

# The test data size is set at 20%. 
X_train, X_test, y_train, y_test = train_test_split(df_norm[independent],df_norm[dependent], test_size=0.2)


Now we will create the tensors. They correspond to the structure used to store information/data in machine learning. 

Also, since the stochastic gradient descent computes the gradient on a subset of the training data, as opposed to the entire training dataset, we need to specify a batch size. Therefore, to avoid generalization, we will use a small batch size of 50. 

In [320]:
BATCH_SIZE = 50
# Tensor for the training data and the value type is float 
X = torch.tensor(X_train.values, dtype=torch.float)
y = torch.tensor(y_train.values, dtype=torch.int64)

train_ds = torch.utils.data.TensorDataset(X,y)

# Tensor for the testing data and the value type is float
X = torch.tensor(X_test.values, dtype=torch.float)
y = torch.tensor(y_test.values, dtype=torch.int64)

test_ds = torch.utils.data.TensorDataset(X,y)

# We shuffle the training dataset
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE,shuffle=True)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)


# Network architecture

The architecture of the neural network consists of 2 hidden layers with 50 neurons. The activation function used is ReLU except for the output layer we use the sigmoid function since the output is binary.  

In [321]:
class MyANN(nn.Module):
  
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(4, 50) 
    self.av1 = nn.ReLU()   
    self.fc2 = nn.Linear(50, 50)    
    self.av2 = nn.ReLU()
    self.fc3 = nn.Linear(50, 50) 
    self.av3 = nn.ReLU() 
    self.fc4 = nn.Linear(50, 50) 
    self.av4 = nn.ReLU()
    self.fc5 = nn.Linear(50, 2) 
    self.av5 = nn.Sigmoid()  
    return


  def forward(self, x):
    x = self.fc1(x)
    x = self.av1(x)
    x = self.fc2(x)
    x = self.av2(x)
    x = self.fc3(x)
    x = self.av3(x)
    x = self.fc4(x)
    x = self.av4(x)
    x = self.fc5(x)
    x = self.av5(x)
    return nn.functional.log_softmax(x, dim=1)

net = MyANN()
print(net)

MyANN(
  (fc1): Linear(in_features=4, out_features=50, bias=True)
  (av1): ReLU()
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (av2): ReLU()
  (fc3): Linear(in_features=50, out_features=50, bias=True)
  (av3): ReLU()
  (fc4): Linear(in_features=50, out_features=50, bias=True)
  (av4): ReLU()
  (fc5): Linear(in_features=50, out_features=2, bias=True)
  (av5): Sigmoid()
)


# Training

At the training level the optimizer used is Adam. The learning rate was initially set to 0.001 and the number of epochs to 200.

In [322]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
for epocs in range(200):
  for data in train_dl:
    X, y = data
    optimizer.zero_grad() # clear gradient information.
    output = net(X.view(-1, 4))
    loss = nn.functional.nll_loss(output, y)
    loss.backward() # do back-propagation step
  optimizer.step() # tell optimizer that you finished batch/iteration.
  print(loss.data)

tensor(0.6932)
tensor(0.6924)
tensor(0.6918)
tensor(0.6913)
tensor(0.6901)
tensor(0.6916)
tensor(0.6873)
tensor(0.6864)
tensor(0.6818)
tensor(0.6868)
tensor(0.6805)
tensor(0.6804)
tensor(0.6764)
tensor(0.6680)
tensor(0.6772)
tensor(0.6662)
tensor(0.6759)
tensor(0.6712)
tensor(0.6786)
tensor(0.6767)
tensor(0.6562)
tensor(0.6824)
tensor(0.6681)
tensor(0.6715)
tensor(0.6799)
tensor(0.6723)
tensor(0.6577)
tensor(0.6748)
tensor(0.6781)
tensor(0.6255)
tensor(0.6707)
tensor(0.6348)
tensor(0.6598)
tensor(0.6722)
tensor(0.6500)
tensor(0.6258)
tensor(0.6545)
tensor(0.6114)
tensor(0.6531)
tensor(0.6763)
tensor(0.6595)
tensor(0.6331)
tensor(0.6443)
tensor(0.5817)
tensor(0.5908)
tensor(0.6107)
tensor(0.5773)
tensor(0.5793)
tensor(0.5633)
tensor(0.5925)
tensor(0.6009)
tensor(0.5844)
tensor(0.5609)
tensor(0.5449)
tensor(0.5773)
tensor(0.5361)
tensor(0.5091)
tensor(0.5433)
tensor(0.5421)
tensor(0.5097)
tensor(0.4823)
tensor(0.4725)
tensor(0.4849)
tensor(0.4835)
tensor(0.4790)
tensor(0.5016)
tensor(0.4

# Testing

In [323]:
total = 0
correct = 0
with torch.no_grad(): 
  for data in test_dl:
    X, y = data
    output = net(X.view(-1, 4))    
    for idx, val in enumerate(output):
      if torch.argmax(val) == y[idx]:
        correct += 1
      total += 1
  print('Accuracy:', round(correct/total, 3))

Accuracy: 0.99
